<a href="https://colab.research.google.com/github/EvertonFernandesJr/pomar-na-mao-processamentos/blob/main/Pomar_na_m%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import xml.etree.ElementTree as ET
import csv
import uuid
from datetime import datetime
import pytz

def kml_to_csv(folder_path: str, file: str):
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}
    filename = file.split('.')[0]
    file_ext = file.split('.')[-1]

    if file_ext == 'kml':
        csv_filename = os.path.splitext(filename)[0] + '.csv'
        csv_path = os.path.join(folder_path, csv_filename)

        try:
            tree = ET.parse(file)
            root = tree.getroot()
            placemarks = root.findall(".//kml:Placemark", ns)

            with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)

                writer.writerow([
                    'id', 'created_at', 'longitude', 'latitude', 'gps_timestamp',
                    'mass', 'harvest', 'description', 'planting_date', 'user_id',
                    'life_of_the_tree', 'stick', 'broken_branch', 'vine_growing',
                    'burnt_branch', 'struck_by_lightning', 'drill', 'anthill',
                    'in_experiment', 'weeds_in_the_basin',
                    'fertilization_or_manuring', 'mites', 'thrips',
                    'empty_collection_box_near', 'variety', 'region','last_work', 'is_dead'
                ])

                for pm in placemarks:
                    name = pm.find('kml:name', ns)
                    coords = pm.find('.//kml:coordinates', ns)

                    if coords is not None:
                        coord_text = coords.text.strip()

                        # Se for polígono, pega só o primeiro vértice
                        if ' ' in coord_text:
                            first_coord = coord_text.split()[0]
                        else:
                            first_coord = coord_text

                        lon, lat, *_ = first_coord.split(',')

                        now = datetime.now(pytz.UTC)

                        writer.writerow([
                            str(uuid.uuid4()),
                            now.strftime('%Y-%m-%d %H:%M:%S.') + f'{int(now.microsecond / 10000):02d}+00',
                            lon.strip(),
                            lat.strip(),
                            1748620471648,
                            None,
                            None,
                            None,
                            None,
                            '680d9ad3-4069-4118-884d-505d18ad553c',
                            None,
                            False, False, False, False, False,
                            False, False, False, False, False,
                            False, False, False,
                            None,
                            'A',
                            None,
                            False
                        ])

            print(f"✔ Saved: {csv_filename}")

        except Exception as e:
            print(f"❌ Failed to process {filename}: {e}")

    else:
        print(f"❌ Invalid file extension: {file_ext}")

kml_to_csv('/content/', 'Arvores.kml')


✔ Saved: Arvores.csv


In [4]:
import csv
from google.colab import files
from shapely.geometry import Point, MultiPoint

#1. Upload dos arquivos─
print(" Faça upload do arquivo de COORDENADAS das árvores (x, y):")
uploaded_trees = files.upload()
trees_file = list(uploaded_trees.keys())[0]

print("\n Faça upload do arquivo de REGIÕES (id, created_at, longitude, latitude, region):")
uploaded_regions = files.upload()
regions_file = list(uploaded_regions.keys())[0]

output_file = "output_" + trees_file

#2. Colunas de saída
OUTPUT_COLUMNS = [
    "id", "created_at", "longitude", "latitude", "gps_timestamp", "mass",
    "harvest", "description", "planting_date", "life_of_the_tree", "stick",
    "broken_branch", "vine_growing", "burnt_branch", "struck_by_lightning",
    "drill", "anthill", "in_experiment", "weeds_in_the_basin",
    "fertilization_or_manuring", "mites", "thrips", "empty_collection_box_near",
    "variety", "region", "is_dead", "is_new", "non_existent", "frost",
    "flowers", "buds", "dehydrated", "updated_at"
]

BOOLEAN_FALSE_FIELDS = {
    "stick", "broken_branch", "vine_growing", "burnt_branch",
    "struck_by_lightning", "drill", "anthill", "in_experiment",
    "weeds_in_the_basin", "fertilization_or_manuring", "mites", "thrips",
    "empty_collection_box_near", "is_dead", "is_new", "non_existent",
    "frost", "flowers", "buds", "dehydrated"
}

#3. Carrega regiões e constrói polígonos (convex hull)
print("\n Construindo polígonos das regiões...")

region_points = {}  # { "A": [(lon, lat), ...], ... }

with open(regions_file, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        region = row['region'].strip()
        lon = float(row['longitude'].strip())
        lat = float(row['latitude'].strip())
        region_points.setdefault(region, []).append((lon, lat))

# Constrói o convex hull de cada região
region_polygons = {}
for region, points in region_points.items():
    if len(points) >= 3:
        region_polygons[region] = MultiPoint(points).convex_hull
    elif len(points) == 2:
        region_polygons[region] = MultiPoint(points).convex_hull  # linha
    else:
        region_polygons[region] = Point(points[0]).buffer(0.0001)  # ponto único

print(f"{len(region_polygons)} regiões carregadas: {sorted(region_polygons.keys())}")

#4. Função para encontrar a região de um ponto
def find_region(lon, lat):
    pt = Point(lon, lat)
    for region, polygon in region_polygons.items():
        if polygon.contains(pt):
            return region
    # Se não estiver dentro de nenhum, retorna a região mais próxima
    min_dist = float('inf')
    closest = None
    for region, polygon in region_polygons.items():
        dist = polygon.distance(pt)
        if dist < min_dist:
            min_dist = dist
            closest = region
    return closest  # ou retorne None se preferir deixar em branco

#5. Detecta colunas X e Y no arquivo de árvores
with open(trees_file, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    orig_fields = reader.fieldnames

x_col = next((c for c in orig_fields if c.strip().lower() in ('x', 'longitude', 'lon', 'lng')), None)
y_col = next((c for c in orig_fields if c.strip().lower() in ('y', 'latitude', 'lat')), None)

if not x_col or not y_col:
    raise ValueError(f"Colunas X/Y não encontradas. Colunas disponíveis: {orig_fields}")

print(f"\n Mapeando '{x_col}' → longitude  |  '{y_col}' → latitude")

#6. Conversão
print("Processando árvores")

with open(trees_file, newline='', encoding='utf-8') as infile, \
     open(output_file, 'w', newline='', encoding='utf-8') as outfile:

    reader = csv.DictReader(infile)
    writer = csv.DictWriter(outfile, fieldnames=OUTPUT_COLUMNS)
    writer.writeheader()

    for i, row in enumerate(reader, start=1):
        lon = float(row.get(x_col, '0').strip())
        lat = float(row.get(y_col, '0').strip())

        region = find_region(lon, lat)

        new_row = {col: None for col in OUTPUT_COLUMNS}
        new_row["id"]        = i
        new_row["longitude"] = lon
        new_row["latitude"]  = lat
        new_row["mass"]      = 0
        new_row["region"]    = region

        for field in BOOLEAN_FALSE_FIELDS:
            new_row[field] = False

        writer.writerow(new_row)

print(f"Conversão concluída! {i} árvores processadas.")

#7. Download do resultado
files.download(output_file)

 Faça upload do arquivo de COORDENADAS das árvores (x, y):


Saving TestePomar7.csv to TestePomar7 (2).csv

 Faça upload do arquivo de REGIÕES (id, created_at, longitude, latitude, region):


Saving regions_rows.csv to regions_rows.csv

 Construindo polígonos das regiões...
12 regiões carregadas: ['A', 'B', 'C', 'D', 'E', 'F', 'FEAGRI', 'G', 'H', 'I', 'J', 'Z']

 Mapeando 'X' → longitude  |  'Y' → latitude
Processando árvores
Conversão concluída! 5623 árvores processadas.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import xml.etree.ElementTree as ET
import csv
import uuid
from datetime import datetime
import pytz

def kml_to_csv(folder_path: str, file: str):
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}
    filename = file.split('.')[0]
    file_ext = file.split('.')[-1]

    if file_ext == 'kml':
        csv_filename = os.path.splitext(filename)[0] + '.csv'
        csv_path = os.path.join(folder_path, csv_filename)

        try:
            tree = ET.parse(file)
            root = tree.getroot()
            placemarks = root.findall(".//kml:Placemark", ns)

            with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)

                writer.writerow([
                    'id', 'created_at', 'longitude', 'latitude', 'gps_timestamp',
                    'mass', 'harvest', 'description', 'planting_date', 'user_id',
                    'life_of_the_tree', 'stick', 'broken_branch', 'vine_growing',
                    'burnt_branch', 'struck_by_lightning', 'drill', 'anthill',
                    'in_experiment', 'weeds_in_the_basin',
                    'fertilization_or_manuring', 'mites', 'thrips',
                    'empty_collection_box_near', 'variety', 'region', 'last_work', 'is_dead'
                ])

                for pm in placemarks:
                    name = pm.find('kml:name', ns)
                    coords = pm.find('.//kml:coordinates', ns)

                    if coords is not None:
                        coord_text = coords.text.strip()
                        lon, lat, *_ = coord_text.split(',')
                        zona = name.text[-1]
                        if name.text[-1].islower():
                          zona = 'FEAGRI'


                        now = datetime.now(pytz.UTC)

                        writer.writerow([
                            str(uuid.uuid4()),
                            now.strftime('%Y-%m-%d %H:%M:%S.') + f'{int(now.microsecond / 10000):02d}+00',
                            lon.strip(),
                            lat.strip(),
                            1748620471648,
                            100,
                            '2024',
                            name.text if name is not None else '',
                            now.strftime('%Y-%m-%d %H:%M:%S.') + f'{int(now.microsecond / 10000):02d}+00',
                            '680d9ad3-4069-4118-884d-505d18ad553c',
                            'Primeira',
                            False, False, False, False, False,
                            False, False, False, False, False,
                            False, False, False,
                            'Clássica (Bengal)',
                            zona
                        ])

            print(f"✔ Saved: {csv_filename}")

        except Exception as e:
            print(f"❌ Failed to process {filename}: {e}")

    else:
        print(f"❌ Invalid file extension: {file_ext}")

# Exemplo de uso:
kml_to_csv('/content/', 'Arvores.kml')


✔ Saved: Zonas.csv
